# Using 'open virtual dataset' capability to work with TEMPO Level 3 data

--- 


## Summary

In this tutorial, we will use the `earthaccess.open_virtual_mfdataset()` function to open a year's worth of granules from the Nitrogen Dioxide ($NO_2$) Level-3 data collection of the [TEMPO air quality mission (link)](https://asdc.larc.nasa.gov/project/TEMPO). We will then calculate means for a subset of the data and visualize the results.

## Prerequisites

- **AWS US-West-2 Environment:** This tutorial has been designed to run in an AWS cloud compute instance in AWS region us-west-2. However, if you want to run it from your laptop or workstation, everything should work just fine but without the speed benefits of in-cloud access.

- **Earthdata Account:** A (free!) Earthdata Login account is required to access data from the NASA Earthdata system. Before requesting TEMPO data, we first need to set up our Earthdata Login authentication, as described in the Earthdata Cookbook's [earthaccess tutorial (link)](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/tutorials/earthaccess-demo.html).

- **Packages:**

  - `cartopy`
  - `dask`
  - `earthaccess` version 0.14.0 or greater
  - `matplotlib`
  - `numpy`
  - `xarray`
## Setup

In [ ]:
import cartopy.crs as ccrs
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from dask.diagnostics import ProgressBar
from matplotlib import rcParams

%config InlineBackend.figure_format = 'jpeg'
rcParams["figure.dpi"] = (
    80  # Reduce figure resolution to keep the saved size of this notebook low.
)

pbar = ProgressBar()
pbar.register()  # Set the ProgressBar to indicate progress for the minutes-long open steps.

## Login using the Earthdata Login

In [ ]:
auth = earthaccess.login()  # earthaccess.system.UAT)

if not auth.authenticated:
    auth.login(
        strategy="interactive", persist=True
    )  # ask for credentials and persist them in a .netrc file

print(earthaccess.__version__)

## Search for data granules

We search for TEMPO Nitrogen Dioxide ($NO_2$) data for a year-long period (note: times are in UTC) betwee January, 2024 and 2025.

In [ ]:
results = earthaccess.search_data(
    short_name="TEMPO_NO2_L3",
    version="V03",
    temporal=("2024-01-11 12:00", "2025-01-11 12:00"),
)
print(f"Number of results: {len(results)}")

## Opening Virtual Multifile Datasets

First we set the argument options to be used by `earthaccess.open_virtual_mfdataset`.

<div class="alert alert-block alert-info">

`load` argument considerations:

- `load=True` works.  Within `earthaccess.open_virtual_mfdataset`, a temporary virtual reference file (a "virtual dataset") is created and then immediately loaded with kerchunk. This is because the function assumes the user is making this request for the first time and the combined manifest file needs to be generated first. In the future, however, `earthaccess.open_virtual_mfdataset` may provide a way to save the combined manifest file, at which point you could then avoid repeating these steps, and proceed directly to loading with kerchunk/virtualizarr.
- `load=False` results in `KeyError: "no index found for coordinate 'longitude'"` because it creates `ManifestArray`s without indexes (see the [earthaccess documentation here (link)](https://github.com/nsidc/earthaccess/blob/7f5fe5d2e42343b6d7948338255cf9bb8cdb2775/earthaccess/dmrpp_zarr.py#L36C456-L36C502))
</div>

In [ ]:
open_options = {
    "access": "direct",
    "load": True,
    "concat_dim": "time",
    "coords": "minimal",
    "compat": "override",
    "combine_attrs": "override",
    "parallel": True,
}

Because TEMPO data are processed and archived in a netCDF4 format using a group hierarchy,
we open each group – i.e., 'root', 'product', and 'geolocation' – and then afterwards merge them together.

In [ ]:
result_root = earthaccess.open_virtual_mfdataset(granules=results, **open_options)
result_product = earthaccess.open_virtual_mfdataset(
    granules=results, group="product", **open_options
)
result_geolocation = earthaccess.open_virtual_mfdataset(
    granules=results, group="geolocation", **open_options
)

Merge root groups with subgroups.

In [ ]:
result_merged = xr.merge([result_root, result_product, result_geolocation])
result_merged

## Temporal Mean - a map showing an annual average

In [ ]:
temporal_mean_ds = (
    result_merged.sel({"longitude": slice(-78, -74), "latitude": slice(35, 39)})
    .where(result_merged["main_data_quality_flag"] == 0)
    .mean(dim=("time"))
)
temporal_mean_ds

In [ ]:
mean_vertical_column_trop = temporal_mean_ds["vertical_column_troposphere"].compute()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

mean_vertical_column_trop.squeeze().plot.contourf(ax=ax)
ax.coastlines()
ax.gridlines(
    draw_labels=True,
    dms=True,
    x_inline=False,
    y_inline=False,
)

## Spatial mean - a time series of area averages

In [ ]:
spatial_mean_ds = (
    result_merged.sel({"longitude": slice(-78, -74), "latitude": slice(35, 39)})
    .where(result_merged["main_data_quality_flag"] == 0)
    .mean(dim=("longitude", "latitude"))
)
spatial_mean_ds

In [ ]:
spatial_mean_vertical_column_trop = spatial_mean_ds[
    "vertical_column_troposphere"
].compute()

In [ ]:
spatial_mean_vertical_column_trop.plot()
plt.show()

## Single scan subset

In [ ]:
subset_ds = result_merged.sel(
    {
        "longitude": slice(-78, -74),
        "latitude": slice(35, 39),
        "time": slice(
            np.datetime64("2024-01-11T13:00:00"), np.datetime64("2024-01-11T14:00:00")
        ),
    }
).where(result_merged["main_data_quality_flag"] == 0)
subset_ds

In [ ]:
subset_vertical_column_trop = subset_ds["vertical_column_troposphere"].compute()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

subset_vertical_column_trop.squeeze().plot.contourf(ax=ax)
ax.coastlines()
ax.gridlines(
    draw_labels=True,
    dms=True,
    x_inline=False,
    y_inline=False,
)
plt.show()